In [9]:
from kafka import KafkaProducer
import time
import csv
import binascii
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [1]:
!pip install kafka-python

     |████████████████████████████████| 256kB 12.1MB/s 


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
PATH = "/content/drive/My Drive/BTFEEDleftmenu.csv"
df = pd.read_csv(PATH)
df.shape

(31808404, 6)

In [10]:
df['timestamp'] = df['event_start_epoc'].apply(lambda ts: datetime.fromtimestamp(ts))  
df

,response_time,event_start_ms,event_start_epoc,year,month,day,timestamp
0,8.674542,564,1544956946,2018,12,16,2018-12-16 10:42:26
1,8.994416,388,1544956946,2018,12,16,2018-12-16 10:42:26
2,10.833192,493,1544956945,2018,12,16,2018-12-16 10:42:25
3,18.026684,315,1544956945,2018,12,16,2018-12-16 10:42:25
4,10.089430,32,1544956945,2018,12,16,2018-12-16 10:42:25
...,...,...,...,...,...,...,...
31808399,195.648865,495,1534143785,2018,8,13,2018-08-13 07:03:05
31808400,27.104675,686,1534143785,2018,8,13,2018-08-13 07:03:05
31808401,14.812492,707,1534143785,2018,8,13,2018-08-13 07:03:05
31808402,11.833150,765,1534143785,2018,8,13,2018-08-13 07:03:05


In [11]:
df.head(2000).to_csv('operational_log.csv', index=True)

In [12]:
PATH = "operational_log.csv"
df_log = pd.read_csv(PATH)
df_log.head()

,Unnamed: 0,response_time,event_start_ms,event_start_epoc,year,month,day,timestamp
0,0,8.674542,564,1544956946,2018,12,16,2018-12-16 10:42:26
1,1,8.994416,388,1544956946,2018,12,16,2018-12-16 10:42:26
2,2,10.833192,493,1544956945,2018,12,16,2018-12-16 10:42:25
3,3,18.026684,315,1544956945,2018,12,16,2018-12-16 10:42:25
4,4,10.089430,32,1544956945,2018,12,16,2018-12-16 10:42:25


In [13]:
# producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'], key_serializer=lambda v: v.encode('utf-8'), value_serializer=lambda v: json.dumps(v).encode('utf-8'), api_version=(0, 10, 1))
producer = KafkaProducer(bootstrap_servers=['localhost:9092','127.0.0.1:9092'],value_serializer=lambda v: v.encode('utf-8'), api_version=(2,0,2))
def sendRowData(data,key,topic):
    if type(topic) == bytes:
      topic = topic.decode('utf-8')
    producer.send(topic, value=data, key=key)

In [14]:
csvfile = open('operational_log.csv', 'r')
fieldnames = ["id", "response_time", "event_start_ms", "event_start_epoc", "year", "month", "day", "timestamp"]
reader = csv.DictReader(csvfile,fieldnames)
next(reader)
for row in reader:    
    print(row)
    print(json.dumps(row))
    print(json.dumps(row).encode('utf-8'))
    break

OrderedDict([('id', '0'), ('response_time', '8.674541994929314'), ('event_start_ms', '564'), ('event_start_epoc', '1544956946'), ('year', '2018'), ('month', '12'), ('day', '16'), ('timestamp', '2018-12-16 10:42:26')])
{"id": "0", "response_time": "8.674541994929314", "event_start_ms": "564", "event_start_epoc": "1544956946", "year": "2018", "month": "12", "day": "16", "timestamp": "2018-12-16 10:42:26"}
b'{"id": "0", "response_time": "8.674541994929314", "event_start_ms": "564", "event_start_epoc": "1544956946", "year": "2018", "month": "12", "day": "16", "timestamp": "2018-12-16 10:42:26"}'


In [ ]:
value_struct = {"schema":
               {"type":"struct",
                "fields":[{"type":"int64","optional":False,"field":"id"},
                          {"type":"float64","optional":False,"field":"response_time"},
                          {"type":"int64","optional":False,"field":"event_start_ms"},
                          {"type":"int64","optional":False,"field":"event_start_epoc"},
                          {"type":"int64","optional":False,"field":"year"},
                          {"type":"int64","optional":False,"field":"month"},
                          {"type":"int64","optional":False,"field":"day"},
                          {"type":"string","optional":False,"field":"timestamp"}],
                "optional":False,
                "name":"hu.elte.anomalydetection.OperationalLog"},
               "payload":{}}
csvfile = open('operational_log.csv', 'r')
fieldnames = ["id", "response_time", "event_start_ms", "event_start_epoc", "year", "month", "day", "timestamp"]
reader = csv.DictReader(csvfile, fieldnames)
next(reader)
for row in reader:    
    value_struct["payload"] = json.loads(json.dumps(row))
    sendRowData(value_struct, row['id'], 'operational-log-data')